In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert data to numpy arrays
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D

# Initialize the scalers
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Fit the scaler to the training data and transform both training and test data
x_data_scaled = scaler_x.fit_transform(x_data)
y_data_scaled = scaler_y.fit_transform(y_data)

# Convert scaled data to Tensors
x_feature_tensors = torch.tensor(x_data_scaled, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data_scaled, dtype=torch.float32)

# Split the training data into training and temporary sets
x_train, x_temp, y_train, y_temp = train_test_split(x_feature_tensors, y_feature_tensors, test_size=0.2, random_state=42)

# Split the temporary set into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Convert scaled labels back to numpy arrays
y_train = y_train.numpy()
y_val = y_val.numpy()
y_test = y_test.numpy()



# Now you can use x_train_scaled, x_val, x_test, y_train_scaled, y_val_scaled, y_test_scaled for your model training


In [3]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train) - window_size
batch_size = 1
input_size = x_train.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [4]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Assuming x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled, x_test_scaled, y_test_scaled,
# input_size, output_size, test_window_size, scaler are available


# Hyperparameter search space
search_space = {
    'learning_rate': [0.00005],
    'window_size': [ 5],
    'hidden_dim': [256],
    'n_layers': [11],
    'batch_evaluation_frequency': [4]
}

#Best Parameters: {'learning_rate': 5e-05, 'window_size': 5, 'hidden_dim': 256, 'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 0.41702309250831604}
#              'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 6463.20556640625}


# Number of random search iterations
num_iterations = 10

best_params = None

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    # Test set evaluation
    with torch.no_grad():
        x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)
        y_test_window = torch.tensor(y_test[:params['window_size']], dtype=torch.float32)
        x_test_window = x_test_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        test_outputs, _ = model(x_test_window, hidden)
        test_loss = criterion(test_outputs, y_test_window)

    print(f"Test Loss: {test_loss.item()}")

    # Update the best_params if the current model performs better on the test set
    if best_params is None:
        best_params = params
        best_params['test_loss'] = test_loss.item()
    elif test_loss < best_params['test_loss']:
        best_params = params
        best_params['test_loss'] = test_loss.item()

print(f"Best Parameters: {best_params}")

C:\Users\pitr7\AppData\Local\Temp\ipykernel_7532\3789981942.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_7532\3789981942.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_val_window = torch.tensor(x_val[

Iteration 1000, Train Loss: 1.0490267276763916, Validation Loss: 2.2497267723083496
Iteration 2000, Train Loss: 2.07322359085083, Validation Loss: 2.2350287437438965
Iteration 3000, Train Loss: 0.23972375690937042, Validation Loss: 2.2370657920837402
Iteration 4000, Train Loss: 0.3800167441368103, Validation Loss: 2.2179744243621826
Iteration 5000, Train Loss: 0.30551591515541077, Validation Loss: 2.2206883430480957
Iteration 6000, Train Loss: 0.17002463340759277, Validation Loss: 2.2246057987213135
Iteration 7000, Train Loss: 0.23032350838184357, Validation Loss: 2.2297072410583496
Iteration 8000, Train Loss: 0.33546438813209534, Validation Loss: 2.2222461700439453


C:\Users\pitr7\AppData\Local\Temp\ipykernel_7532\3789981942.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)


Test Loss: 0.4182278513908386
Iteration 1000, Train Loss: 1.0481255054473877, Validation Loss: 2.2489237785339355
Iteration 2000, Train Loss: 2.071110486984253, Validation Loss: 2.2342686653137207
Iteration 3000, Train Loss: 0.24180194735527039, Validation Loss: 2.235933780670166
Iteration 4000, Train Loss: 0.37913402915000916, Validation Loss: 2.2184417247772217
Iteration 5000, Train Loss: 0.3061540126800537, Validation Loss: 2.2203803062438965
Iteration 6000, Train Loss: 0.17040665447711945, Validation Loss: 2.2243080139160156
Iteration 7000, Train Loss: 0.23221734166145325, Validation Loss: 2.2287352085113525
Iteration 8000, Train Loss: 0.3347059488296509, Validation Loss: 2.222489595413208
Test Loss: 0.4165410101413727
Iteration 1000, Train Loss: 1.0487163066864014, Validation Loss: 2.249377489089966
Iteration 2000, Train Loss: 2.0708460807800293, Validation Loss: 2.2341809272766113
Iteration 3000, Train Loss: 0.24090614914894104, Validation Loss: 2.2364439964294434
Iteration 4000,

In [23]:
# Assuming 'x_test' is your test dataset, and you want to predict the next day after the last day in the test set
last_day_index = -1  # Index of the last day in the test set
next_day_features = x_test[last_day_index].reshape(1, 1, input_size)

In [24]:
# Assuming you have a trained model named 'model'

# Set the model in evaluation mode
model.eval()

# Initialize a list to store the predicted values
predicted_values = []

# Use the last window of your training data to initialize the prediction
initial_window = torch.tensor(x_train[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)
hidden = model.init_hidden(1)
outputs, hidden = model(initial_window, hidden)

# Iteratively predict the next day and update the window
for _ in range(num_future_days):
    # Assuming 'next_day_features' contains the features for the next day
    next_day_features = torch.tensor(next_day_features, dtype=torch.float32).view(1, 1, input_size)

    # Make a prediction for the next day
    with torch.no_grad():
        prediction, _ = model(next_day_features, hidden)

    # Update the window by removing the first day and adding the predicted value
    initial_window = torch.cat((initial_window[:, 1:, :], next_day_features), dim=1)

    # Update the hidden state for the next prediction
    hidden = model.init_hidden(1)

    # Append the predicted value to the list
    predicted_values.append(prediction.item())

# Convert the predicted values back to the original scale if you used scaling
predicted_values = scaler_y.inverse_transform(np.array(predicted_values).reshape(-1, 1))

# Print or use the predicted values as needed
print(predicted_values)


C:\Users\pitr7\AppData\Local\Temp\ipykernel_7532\231502711.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  initial_window = torch.tensor(x_train[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_7532\231502711.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_day_features = torch.tensor(next_day_features, dtype=torch.float32).view(1, 1, input_size)


[[352.3540612]
 [130.973271 ]
 [130.973271 ]
 [130.973271 ]]


In [5]:
# Assuming 'data' is your original DataFrame containing the time series data
print(data.tail(5))


             date    open    high     low   close      volume  adjusted_close  \
10742  2023-07-25  193.33  194.44  192.92  193.62  37283200.0        193.3589   
10743  2023-07-26  193.67  195.64  193.32  194.50  47471900.0        194.2377   
10744  2023-07-27  196.02  197.20  192.55  193.22  47460200.0        192.9594   
10745  2023-07-28  194.67  196.63  194.14  195.83  48291400.0        195.5659   
10746  2023-07-31  196.06  196.49  195.26  196.45  38824100.0        196.1851   

       change_percent  avg_vol_20d  
10742            0.45   52369165.0  
10743            0.45   52206220.0  
10744           -0.66   52018390.0  
10745            1.35   52115595.0  
10746            0.32   49803320.0  


Orignial rolling window

In [6]:
# Assuming 'window_size', model, scaler_x, scaler_y, input_size are defined
params['window_size'] = 5  # Adjust window_size as needed

# Initialize the rolling window with the last values from the training set, excluding 'close'
initial_rolling_window = data.iloc[-params['window_size']:][['open', 'high', 'low', 'volume', 'adjusted_close',
                                                             'change_percent', 'avg_vol_20d']].values
rolling_window_original = initial_rolling_window  # Save for printing later

# Set the correct input size based on the number of features in your data
input_size = initial_rolling_window.shape[1]

# Print the initial rolling window values and dimensions
print("Initial Rolling Window (Original Values):\n", rolling_window_original)

Initial Rolling Window (Original Values):
 [[ 1.9333000e+02  1.9444000e+02  1.9292000e+02  3.7283200e+07
   1.9335890e+02  4.5000000e-01  5.2369165e+07]
 [ 1.9367000e+02  1.9564000e+02  1.9332000e+02  4.7471900e+07
   1.9423770e+02  4.5000000e-01  5.2206220e+07]
 [ 1.9602000e+02  1.9720000e+02  1.9255000e+02  4.7460200e+07
   1.9295940e+02 -6.6000000e-01  5.2018390e+07]
 [ 1.9467000e+02  1.9663000e+02  1.9414000e+02  4.8291400e+07
   1.9556590e+02  1.3500000e+00  5.2115595e+07]
 [ 1.9606000e+02  1.9649000e+02  1.9526000e+02  3.8824100e+07
   1.9618510e+02  3.2000000e-01  4.9803320e+07]]


In [7]:
# Assuming 'model', 'scaler_x', 'scaler_y', 'input_size' are defined

# Set the window size for prediction
prediction_window_size = 1  # Predicting for the next day

# Initialize the rolling window with the last values from the training set
initial_rolling_window = data.iloc[-params['window_size']:][['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values

# Print the initial rolling window values and dimensions
print("Initial Rolling Window (Original Values):\n", initial_rolling_window)


# Check the number of features in the rolling window
if initial_rolling_window.shape[1] != input_size:
    # Print or handle the inconsistency
    print("Number of features in the rolling window does not match the expected input_size.")

# Scale the input features using the trained scaler
scaled_input = scaler_x.transform(initial_rolling_window)

# Convert the scaled input features to a PyTorch tensor
input_tensor = torch.tensor(scaled_input, dtype=torch.float32).view(1, params['window_size'], input_size)

# Initialize the hidden state
hidden = model.init_hidden(1)

# Perform the prediction
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    predicted_close_scaled, _ = model(input_tensor, hidden)

# Inverse transform the predicted close value to the original scale
predicted_close_original = scaler_y.inverse_transform(predicted_close_scaled.numpy())

# Print the predicted close value
print("Predicted Close Value for the Next Day (Original Scale):", predicted_close_original)


Initial Rolling Window (Original Values):
 [[ 1.9333000e+02  1.9444000e+02  1.9292000e+02  3.7283200e+07
   1.9335890e+02  4.5000000e-01  5.2369165e+07]
 [ 1.9367000e+02  1.9564000e+02  1.9332000e+02  4.7471900e+07
   1.9423770e+02  4.5000000e-01  5.2206220e+07]
 [ 1.9602000e+02  1.9720000e+02  1.9255000e+02  4.7460200e+07
   1.9295940e+02 -6.6000000e-01  5.2018390e+07]
 [ 1.9467000e+02  1.9663000e+02  1.9414000e+02  4.8291400e+07
   1.9556590e+02  1.3500000e+00  5.2115595e+07]
 [ 1.9606000e+02  1.9649000e+02  1.9526000e+02  3.8824100e+07
   1.9618510e+02  3.2000000e-01  4.9803320e+07]]
Predicted Close Value for the Next Day (Original Scale): [[116.38846]]
